# Similarity score comparison

In [1]:
!pip install matchms
!pip install pubchempy
!pip install rdkit-pypi
!pip install spec2vec --user

In [21]:
import os
import numpy as np
from matchms.importing import load_from_mgf
import matplotlib.pyplot as plt
import pubchempy
import pandas as pd
from itertools import compress
from rdkit import Chem
from rdkit.Chem import Draw

In [22]:
from matchms import calculate_scores
from matchms.similarity import CosineGreedy
from matchms.similarity import ModifiedCosine

In [23]:
import gensim
from spec2vec import Spec2Vec
from matchms.filtering import default_filters
from matchms.filtering import normalize_intensities

In [24]:
path_data = "C:/Users/naake/Documents/GitHub/SimilarityHackathon/data/"
file_mgf_reference = os.path.join(path_data, "ex_spectra_stds_NEG_scans_50.mgf")
reference_spectra = list(load_from_mgf(file_mgf_reference))

In [25]:
file_mgf_query = os.path.join(path_data, "pseudo_query_neg_v1.mgf")
query_spectra = list(load_from_mgf(file_mgf_query))

# Expand grid function

Define a function that calculates all possible combinations of parameters. Iterate over the data frame in the following.

In [138]:
from itertools import product
import pandas as pd

def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())


combinations_cosine = {
    'tolerance': [0.005, 0.01], 
    'mz_power': [0, 0.5, 1, 2], 
    'intensity_power': [0, 0.5, 1, 2]}
combinations_modified_cosine = {
    'tolerance': [0.005, 0.01], 
    'mz_power': [0, 0.5, 1, 2], 
    'intensity_power': [0, 0.5, 1, 2]}
combinations_neutralloss_cosine = {
    'tolerance': [0.005, 0.01], 
    'mz_power': [0, 0.5, 1, 2], 
    'intensity_power': [0, 0.5, 1, 2],
    'ignore_peaks_above_precursor': [True, False]}
combinations_spec2vec = {
    'intensity_weighting_power': [0, 0.5, 1, 2],
    'allowed_missing_percentage': [1, 5, 10, 20]}

## expand the grids for the parameter spaces
combinations_cosine = expand_grid(combinations_cosine)
combinations_modified_cosine = expand_grid(combinations_modified_cosine)
combinations_neutralloss_cosine = expand_grid(combinations_neutralloss_cosine)
combinations_spec2vec = expand_grid(combinations_spec2vec)

## Spec2Vec

In [114]:
path_model = "C:/Users/naake/Documents/GitHub/SimilarityHackathon/data/"
filename_model = "spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model"
filename = os.path.join(path_model, filename_model)
model = gensim.models.Word2Vec.load(filename)

In [115]:
def peak_processing(spectrum):
    spectrum = default_filters(spectrum)
    spectrum = normalize_intensities(spectrum)
    return spectrum

reference_s2v_spectra = [peak_processing(s) for s in reference_spectra]
query_s2v_spectra = [peak_processing(s) for s in query_spectra if len(s.peaks) != 0]

2023-01-17 17:25:20,308:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 319.2280252 with new one: 319.2280252
2023-01-17 17:25:20,337:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 319.2280252 with new one: 319.2280252
2023-01-17 17:25:20,358:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 319.2280252 with new one: 319.2280252
2023-01-17 17:25:20,383:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 319.2280252 with new one: 319.2280252
2023-01-17 17:25:20,409:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 319.2280252 with new one: 319.2280252
2023-01-17 17:25:20,429:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 313.23925153 with new one: 313.23925153
2023-01-17 17:25:20,445:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 313.23925153 with new one: 313.23925153
2023-01-17 17:25:20,470:WARNING:matchms:interpret_pepmass:Overwriting existing precurs

2023-01-17 17:25:22,046:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 317.21236411 with new one: 317.21236411
2023-01-17 17:25:22,073:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 317.2119978 with new one: 317.2119978
2023-01-17 17:25:22,099:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 295.22835622 with new one: 295.22835622
2023-01-17 17:25:22,127:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 295.22835622 with new one: 295.22835622
2023-01-17 17:25:22,155:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 295.22835622 with new one: 295.22835622
2023-01-17 17:25:22,181:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 295.22835622 with new one: 295.22835622
2023-01-17 17:25:22,211:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 295.22835622 with new one: 295.22835622
2023-01-17 17:25:22,236:WARNING:matchms:interpret_pepmass:Overwriting existing

2023-01-17 17:25:23,571:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 495.29588685 with new one: 495.29588685
2023-01-17 17:25:23,584:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 495.29588685 with new one: 495.29588685
2023-01-17 17:25:23,611:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 495.29588685 with new one: 495.29588685
2023-01-17 17:25:23,639:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 495.29588685 with new one: 495.29588685
2023-01-17 17:25:23,659:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 367.2487631 with new one: 367.2487631
2023-01-17 17:25:23,685:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 367.2487631 with new one: 367.2487631
2023-01-17 17:25:23,711:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 367.2487631 with new one: 367.2487631
2023-01-17 17:25:23,737:WARNING:matchms:interpret_pepmass:Overwriting existing pre

2023-01-17 17:25:24,931:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 193.05062439 with new one: 193.05062439
2023-01-17 17:25:24,951:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 193.05062439 with new one: 193.05062439
2023-01-17 17:25:24,975:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 193.05062439 with new one: 193.05062439
2023-01-17 17:25:24,994:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 193.05062439 with new one: 193.05062439
2023-01-17 17:25:25,012:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 193.05062439 with new one: 193.05062439
2023-01-17 17:25:25,032:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 153.01934725 with new one: 153.01934725
2023-01-17 17:25:25,058:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 153.01934725 with new one: 153.01934725
2023-01-17 17:25:25,076:WARNING:matchms:interpret_pepmass:Overwriting existi

2023-01-17 17:25:26,260:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 187.13430896 with new one: 187.13430896
2023-01-17 17:25:26,281:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 187.13430896 with new one: 187.13430896
2023-01-17 17:25:26,298:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 187.13430896 with new one: 187.13430896
2023-01-17 17:25:26,317:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 187.13398851 with new one: 187.13398851
2023-01-17 17:25:26,339:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 215.16598408 with new one: 215.16598408
2023-01-17 17:25:26,358:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 215.16598408 with new one: 215.16598408
2023-01-17 17:25:26,383:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 215.16598408 with new one: 215.16598408
2023-01-17 17:25:26,408:WARNING:matchms:interpret_pepmass:Overwriting existi

...

2023-01-17 17:26:12,299:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 509.38267477 with new one: 509.38267477
2023-01-17 17:26:12,318:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 509.3824001 with new one: 509.3824001
2023-01-17 17:26:12,335:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 509.3824001 with new one: 509.3824001
2023-01-17 17:26:12,352:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 509.3824001 with new one: 509.3824001
2023-01-17 17:26:12,377:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 509.38267477 with new one: 509.38267477
2023-01-17 17:26:12,393:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 514.28417658 with new one: 514.28417658
2023-01-17 17:26:12,415:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 514.28417658 with new one: 514.28417658
2023-01-17 17:26:12,435:WARNING:matchms:interpret_pepmass:Overwriting existing pre

2023-01-17 17:26:13,588:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 534.28697695 with new one: 534.28697695
2023-01-17 17:26:13,608:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 534.28532893 with new one: 534.28532893
2023-01-17 17:26:13,627:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 540.33034239 with new one: 540.33034239
2023-01-17 17:26:13,645:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 540.33034239 with new one: 540.33034239
2023-01-17 17:26:13,668:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 540.33034239 with new one: 540.33034239
2023-01-17 17:26:13,690:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 540.33034239 with new one: 540.33034239
2023-01-17 17:26:13,710:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 540.33034239 with new one: 540.33034239
2023-01-17 17:26:13,731:WARNING:matchms:interpret_pepmass:Overwriting existi

2023-01-17 17:26:14,956:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 619.41736569 with new one: 619.41736569
2023-01-17 17:26:14,980:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 619.41760984 with new one: 619.41760984
2023-01-17 17:26:15,001:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 619.41760984 with new one: 619.41760984
2023-01-17 17:26:15,030:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 619.41760984 with new one: 619.41760984
2023-01-17 17:26:15,050:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 633.29841054 with new one: 633.29841054
2023-01-17 17:26:15,075:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 633.29841054 with new one: 633.29841054
2023-01-17 17:26:15,102:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 633.29841054 with new one: 633.29841054
2023-01-17 17:26:15,130:WARNING:matchms:interpret_pepmass:Overwriting existi

2023-01-17 17:26:16,319:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 813.53103261 with new one: 813.53103261
2023-01-17 17:26:16,345:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 815.56696829 with new one: 815.56696829
2023-01-17 17:26:16,367:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 815.56696829 with new one: 815.56696829
2023-01-17 17:26:16,386:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 815.56696829 with new one: 815.56696829
2023-01-17 17:26:16,413:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 815.56696829 with new one: 815.56696829
2023-01-17 17:26:16,435:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 815.56696829 with new one: 815.56696829
2023-01-17 17:26:16,451:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 819.60963929 with new one: 819.60963929
2023-01-17 17:26:16,468:WARNING:matchms:interpret_pepmass:Overwriting existi

2023-01-17 17:26:17,663:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 965.5971646 with new one: 965.5971646
2023-01-17 17:26:17,684:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 965.59631007 with new one: 965.59631007
2023-01-17 17:26:17,701:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 965.5971646 with new one: 965.5971646
2023-01-17 17:26:17,727:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 987.57871591 with new one: 987.57871591
2023-01-17 17:26:17,755:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 987.57871591 with new one: 987.57871591
2023-01-17 17:26:17,781:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 987.57871591 with new one: 987.57871591
2023-01-17 17:26:17,802:WARNING:matchms:interpret_pepmass:Overwriting existing precursor_mz 987.57871591 with new one: 987.57871591
2023-01-17 17:26:17,826:WARNING:matchms:interpret_pepmass:Overwriting existing p

In [139]:
combinations_spec2vec

,intensity_weighting_power,allowed_missing_percentage
0,0.0,1
1,0.0,5
2,0.0,10
3,0.0,20
4,0.5,1
5,0.5,5
6,0.5,10
7,0.5,20
8,1.0,1
9,1.0,5


In [140]:
new_colnames = []
new_cols = []

for index, row in combinations_spec2vec.iterrows():
    spec2vec_similarity = Spec2Vec(model=model,
                               intensity_weighting_power=row["intensity_weighting_power"],
                               allowed_missing_percentage=row["allowed_missing_percentage"])
    spec2vec_scores = calculate_scores(reference_s2v_spectra, query_s2v_spectra, spec2vec_similarity,
                          is_symmetric=False)
    scores = spec2vec_scores.scores
    SCORES = np.zeros(shape=scores.shape)
    for i in range(scores.shape[0]):
        for j in range(scores.shape[1]):
            SCORES[i][j] = scores[i][j]#[0]
    SCORES_df = pd.DataFrame(SCORES)
    
    ## add row- and colnames to data.frame
    SCORES_df.index = [s.metadata["compound_name"] + "_" + s.metadata["collision_energy"] for s in reference_s2v_spectra]
    SCORES_df.columns = [s.metadata["scanindex"] + "_" + s.metadata["collision_energy"] for s in query_s2v_spectra]
    SCORES_df['rows'] = SCORES_df.index

    ## from wide to long
    SCORES_df = pd.melt(SCORES_df, id_vars ='rows', value_vars = list(SCORES_df.columns[:-1]))
    new_column = 'Spec2Vec' + "_" + str(row['intensity_weighting_power']) + "_" + str(row['allowed_missing_percentage'])
    new_colnames.append(new_column)
    new_cols.append(SCORES_df['value'].values)
    
    SCORES_df = SCORES_df.rename(columns={'value': 'Spec2Vec' + "_" + str(row['intensity_weighting_power']) + "_" + str(row['allowed_missing_percentage'])})
    

2023-01-17 17:40:56,849:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,851:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,852:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,854:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,857:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:40:56,974:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,979:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,982:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,986:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:56,989:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:40:57,146:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,154:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,158:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,162:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,166:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:40:57,331:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,334:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,337:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,339:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,350:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:40:57,512:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,525:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,535:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,542:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:40:57,547:WARNING:spec2vec:vector_operations:('Missing percentage (50.

...

2023-01-17 17:41:15,294:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,297:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,299:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,302:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,305:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:41:15,401:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,403:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,405:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,407:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,409:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:41:15,519:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,522:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,524:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,528:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,534:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:41:15,675:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,677:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,681:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,684:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,688:WARNING:spec2vec:vector_operations:('Missing percentage (50.

2023-01-17 17:41:15,855:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,865:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,870:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,874:WARNING:spec2vec:vector_operations:('Missing percentage (50.00%) is above set maximum. An empty vector will be returned.', 'Consider retraining the used model or change the `allowed_missing_percentage`.')
2023-01-17 17:41:15,876:WARNING:spec2vec:vector_operations:('Missing percentage (50.

In [141]:
for i, colname in enumerate(new_colnames):
    SCORES_df[colname] = list(new_cols[i])

In [142]:
SCORES_df

,rows,variable,Spec2Vec_2.0_20.0,Spec2Vec_0.0_1.0,Spec2Vec_0.0_5.0,Spec2Vec_0.0_10.0,Spec2Vec_0.0_20.0,Spec2Vec_0.5_1.0,Spec2Vec_0.5_5.0,Spec2Vec_0.5_10.0,Spec2Vec_0.5_20.0,Spec2Vec_1.0_1.0,Spec2Vec_1.0_5.0,Spec2Vec_1.0_10.0,Spec2Vec_1.0_20.0,Spec2Vec_2.0_1.0,Spec2Vec_2.0_5.0,Spec2Vec_2.0_10.0
0,11-hydroxyeicosatetraenoic acid_10,1_10,-0.088317,NaN,NaN,NaN,NaN,-0.148681,-0.148681,-0.148681,-0.148681,-0.129823,-0.129823,-0.129823,-0.129823,-0.088317,-0.088317,-0.088317
1,11-hydroxyeicosatetraenoic acid_20,1_10,-0.173825,NaN,NaN,NaN,NaN,-0.183356,-0.183356,-0.183356,-0.183356,-0.180468,-0.180468,-0.180468,-0.180468,-0.173825,-0.173825,-0.173825
2,11-hydroxyeicosatetraenoic acid_30,1_10,-0.175747,NaN,NaN,NaN,NaN,-0.202205,-0.202205,-0.202205,-0.202205,-0.187190,-0.187190,-0.187190,-0.187190,-0.175747,-0.175747,-0.175747
3,11-hydroxyeicosatetraenoic acid_40,1_10,-0.177001,NaN,NaN,NaN,NaN,-0.148220,-0.148220,-0.148220,-0.148220,-0.162778,-0.162778,-0.162778,-0.162778,-0.177001,-0.177001,-0.177001
4,11-hydroxyeicosatetraenoic acid_60,1_10,-0.016073,NaN,NaN,NaN,NaN,-0.051248,-0.051248,-0.051248,-0.051248,-0.039866,-0.039866,-0.039866,-0.039866,-0.016073,-0.016073,-0.016073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562347,Ursodeoxycholate_10,1614_40,-0.090932,-0.166517,-0.166517,-0.166517,-0.166517,-0.094399,-0.094399,-0.094399,-0.094399,-0.092282,-0.092282,-0.092282,-0.092282,-0.090932,-0.090932,-0.090932
1562348,Ursodeoxycholate_20,1614_40,-0.090627,-0.112742,-0.112742,-0.112742,-0.112742,-0.091469,-0.091469,-0.091469,-0.091469,-0.092155,-0.092155,-0.092155,-0.092155,-0.090627,-0.090627,-0.090627
1562349,Ursodeoxycholate_30,1614_40,-0.091939,-0.089974,-0.089974,-0.089974,-0.089974,-0.082289,-0.082289,-0.082289,-0.082289,-0.093102,-0.093102,-0.093102,-0.093102,-0.091939,-0.091939,-0.091939
1562350,Ursodeoxycholate_40,1614_40,-0.089873,0.002466,0.002466,0.002466,0.002466,-0.073208,-0.073208,-0.073208,-0.073208,-0.091563,-0.091563,-0.091563,-0.091563,-0.089873,-0.089873,-0.089873


In [143]:
SCORES_df.to_csv("similarities_Spec2Vec.csv")